In [1]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine

In [2]:
# Import EDU file
csv_file = "C:/Users/admin/HW/project2/Resources/edu.csv"
edu_df = pd.read_csv(csv_file)
edu_df.head()

,country,c_codes,cal_year,gnp,population,edu_prime,edu_second,edu_post_second,edu_lower_second,unemp
0,ARAB WORLD,NaN,2017,16997.50354,414491886.0,NaN,NaN,NaN,NaN,NaN
1,ARAB WORLD,NaN,2016,16794.06495,406452690.0,83.911949,NaN,NaN,NaN,NaN
2,ARAB WORLD,NaN,2015,16366.59640,398304960.0,84.720306,NaN,NaN,NaN,NaN
3,ARAB WORLD,NaN,2014,15895.25148,390043028.0,85.314812,NaN,NaN,NaN,NaN
4,ARAB WORLD,NaN,2013,15453.22069,381702086.0,84.703484,NaN,NaN,NaN,NaN


In [3]:
# Filter years for 2005, 2010 & 2015

# Indicate reference years
years = (2005, 2010, 2015)

# Filter reference years
edu_df = edu_df[edu_df['cal_year'].isin(years)]

# Select columns
edu_df = edu_df[['country','cal_year', 'gnp', 'population', 'edu_prime']]

edu_df


,country,cal_year,gnp,population,edu_prime
2,ARAB WORLD,2015,16366.59640,398304960.0,84.720306
7,ARAB WORLD,2010,13942.73328,356508908.0,85.171822
12,ARAB WORLD,2005,11537.59540,316264728.0,82.122704
60,CARIBBEAN SMALL STATES,2015,15516.02300,7204948.0,NaN
65,CARIBBEAN SMALL STATES,2010,13380.00937,6984096.0,91.299530
...,...,...,...,...,...
15203,ZAMBIA,2010,3000.00000,13850033.0,95.396584
15208,ZAMBIA,2005,2040.00000,12052156.0,80.275414
15256,ZIMBABWE,2015,1790.00000,15777451.0,NaN
15261,ZIMBABWE,2010,1240.00000,14086317.0,NaN


In [4]:
# import HDI File
csv_file = "C:/Users/admin/HW/project2/Resources/human_development_index.csv"
hdi_df = pd.read_csv(csv_file)
hdi_df.head()

,country,hdi_rank_2018,1990,1991,1992,1993,1994,1995,1996,1997,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Afghanistan,170.0,0.298,0.304,0.312,0.308,0.303,0.327,0.331,0.335,...,0.447,0.464,0.465,0.479,0.485,0.488,0.490,0.491,0.493,0.496
1,Albania,69.0,0.644,0.625,0.608,0.611,0.617,0.629,0.639,0.639,...,0.729,0.740,0.759,0.771,0.781,0.787,0.788,0.788,0.789,0.791
2,Algeria,82.0,0.578,0.582,0.589,0.593,0.597,0.602,0.610,0.619,...,0.720,0.730,0.738,0.737,0.746,0.749,0.751,0.755,0.758,0.759
3,Andorra,36.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.830,0.828,0.827,0.849,0.846,0.853,0.850,0.854,0.852,0.857
4,Angola,149.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.508,0.510,0.525,0.537,0.547,0.557,0.565,0.570,0.576,0.574


In [5]:
# Transpose HDI DataFrame

hdi_df = hdi_df.melt(id_vars=['country', 'hdi_rank_2018'],
                     var_name =['cal_year'],
                     value_name ='hdi')

hdi_df

,country,hdi_rank_2018,cal_year,hdi
0,Afghanistan,170.0,1990,0.298
1,Albania,69.0,1990,0.644
2,Algeria,82.0,1990,0.578
3,Andorra,36.0,1990,NaN
4,Angola,149.0,1990,NaN
...,...,...,...,...
5650,Nauru,NaN,2018,NaN
5651,Korea (Democratic People's Rep. of),NaN,2018,NaN
5652,San Marino,NaN,2018,NaN
5653,Somalia,NaN,2018,NaN


In [6]:
# # Filter reference years
years = ("2005", "2010", "2015")

# Filter reference years
hdi_df = hdi_df[hdi_df['cal_year'].isin(years)]

hdi_df

,country,hdi_rank_2018,cal_year,hdi
2925,Afghanistan,170.0,2005,0.410
2926,Albania,69.0,2005,0.702
2927,Algeria,82.0,2005,0.694
2928,Andorra,36.0,2005,0.819
2929,Angola,149.0,2005,0.453
...,...,...,...,...
5065,Nauru,NaN,2015,NaN
5066,Korea (Democratic People's Rep. of),NaN,2015,NaN
5067,San Marino,NaN,2015,NaN
5068,Somalia,NaN,2015,NaN


In [7]:
# Connect to local Database
USERNAME = "postgres"
PASSWORD = "postgres"
DATABASE = "Project_2"
rds_connection_string = f"{USERNAME}:{PASSWORD}@localhost:5432/{DATABASE}"
print(rds_connection_string)
engine = create_engine(f'postgresql://{rds_connection_string}')

postgres:postgres@localhost:5432/Project_2


In [8]:
# Check for tables
engine.table_names()

['hdi', 'edu']

In [9]:
# Use pandas to load csv converted DataFrame into database
edu_df.to_sql(name='edu', con=engine, if_exists='append', index=False)

# check for loaded data
pd.read_sql_query('select * from edu', con=engine).head()

,country,cal_year,gnp,population,edu_prime
0,ARAB WORLD,2015,16366.59640,398304960.0,84.720306
1,ARAB WORLD,2010,13942.73328,356508908.0,85.171822
2,ARAB WORLD,2005,11537.59540,316264728.0,82.122704
3,CARIBBEAN SMALL STATES,2015,15516.02300,7204948.0,NaN
4,CARIBBEAN SMALL STATES,2010,13380.00937,6984096.0,91.299530


In [10]:
# Use pandas to load csv converted DataFrame into database
hdi_df.to_sql(name='hdi', con=engine, if_exists='append', index=False)

In [11]:
# check for loaded data
pd.read_sql_query('select * from hdi', con=engine).head()

,country,hdi_rank_2018,cal_year,hdi
0,Afghanistan,170.0,2005,0.410
1,Albania,69.0,2005,0.702
2,Algeria,82.0,2005,0.694
3,Andorra,36.0,2005,0.819
4,Angola,149.0,2005,0.453
